# Step 4: Final Data Cleaning and Fixes

**Metis Project 2, Andrew Zhou**

We do some final cleaning passes and add some columns to the dataframe.

In [31]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from scipy import stats

In [32]:
mal_info_df = pd.read_pickle("../data/mal_info_df.pickle")
anime_sales_df = pd.read_pickle("../data/anime_sales_df_linked.pickle")

Add the MAL info.

In [33]:
anime_sales_df = pd.concat([anime_sales_df.loc[anime_sales_df.index], mal_info_df], join="inner", axis=1)

In [34]:
anime_sales_df["sales"] = anime_sales_df["gross_1st_rls"].map(lambda x: x[1:].replace(",", "")).astype(float)

Drop certain unknown data.

In [35]:
anime_sales_df = anime_sales_df[anime_sales_df["sales"] != 0]
anime_sales_df = anime_sales_df[anime_sales_df["rating"] != "None"]
anime_sales_df = anime_sales_df[anime_sales_df["duration"] != "Unknown"]

anime_sales_df = anime_sales_df.dropna(subset=["score"])

Modify and typecast some columns and make some new ones.

In [48]:
cols_to_int = ["duration", "episodes", "year", "members"]
anime_sales_df[cols_to_int] = anime_sales_df[cols_to_int].astype(int)

anime_sales_df["broadcast"] = anime_sales_df["broadcast"].apply(lambda x: (None, None) if not x else x)
anime_sales_df[["day", "time"]] = pd.DataFrame(anime_sales_df["broadcast"].tolist(), index=anime_sales_df.index)
anime_sales_df["hour"] = anime_sales_df["time"].apply(lambda x: int(x[:2]) if x else None)
anime_sales_df["runtime"] = anime_sales_df["episodes"]*anime_sales_df["duration"]

Make a binary variable (0 or 1) for each genre and add these variables to the 
dataframe.

In [49]:
genre_list = np.unique(anime_sales_df["genres"].aggregate(sum))
# manually delete 2 columns with scarcely any values
np.delete(genre_list, np.where((genre_list == "Cars")|(genre_list == "Dementia")))

# series of series, each series has columns of genres and 1 if it's of that genre
ser_ser = anime_sales_df["genres"].map(lambda x: pd.Series(1, index=x))
# turn series of series into a df with same indices as original df
genre_df = ser_ser.apply(lambda x: x)
genre_df = genre_df.fillna(0)

anime_sales_df = pd.concat([anime_sales_df, genre_df], axis=1)


Drop certain outliers.

In [51]:
anime_sales_df = anime_sales_df[anime_sales_df["sales"] < 500]
anime_sales_df = anime_sales_df[(np.abs(stats.zscore(anime_sales_df[["score", "duration", "members"]])) < 2).all(axis=1)]

Put sales at the end of all the columns for neatness's sake.

In [52]:
column_list = anime_sales_df.columns.tolist()
column_list.remove("sales")
column_list.append("sales")

anime_sales_df = anime_sales_df[column_list]

In [53]:
len(anime_sales_df)

2

In [54]:
anime_sales_df.to_pickle("../data/anime_sales_df_final.pickle")

In [55]:
anime_sales_df.columns

Index(['title', 'year', 'season_qtr', 'avg_sales', 're_rls', 'total',
       'gross_1st_rls', 'studio', 'source', 'link', 'episodes', 'broadcast',
       'genres', 'duration', 'rating', 'score', 'members', 'favorites', 'day',
       'time', 'hour', 'runtime', 'Game', 'Game', 'Sci-Fi', 'Sci-Fi',
       'Adventure', 'Adventure', 'Mystery', 'Mystery', 'Magic', 'Magic',
       'Fantasy', 'Fantasy', 'Comedy', 'Comedy', 'Shounen', 'Shounen', 'Game',
       'Game', 'Sci-Fi', 'Sci-Fi', 'Adventure', 'Adventure', 'Mystery',
       'Mystery', 'Magic', 'Magic', 'Fantasy', 'Fantasy', 'Comedy', 'Comedy',
       'Shounen', 'Shounen', 'sales'],
      dtype='object')